# Segmentação de vendas entre 2014 - 2015.

### Carregando a base de dados usada para o projeto.

In [ ]:
dados = read.delim(file = '/content/Vendas', header = FALSE, sep = '\t', dec = '.')

### Definindo as colunas ou domínios dos dados para melhor classificação e processamento.

In [ ]:
# Adicionando cabeçalho e interpretando a última coluna como data tirando o ano de compra

colnames(dados) = c('ID_cliente', 'quantia_compra', 'data_compra')

dados$data_compra = as.Date(dados$data_compra, '%Y-%m-%d')

dados$ano_compra = as.numeric(format(dados$date_of_purchase, '%Y'))

dados$dias_desde = as.numeric(difftime(time1 = '2016-01-01', time2 = dados$data_compra, units = 'days'))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


### Importando a biblioteca de SQL e usando comandos SQL para criar uma nova entidade chamada “clientes_2015”.

In [ ]:
install.packages('sqldf')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘plogr’, ‘gsubfn’, ‘proto’, ‘RSQLite’, ‘chron’




In [ ]:
# Invocando a biblioteca para computar indicadores usando SQL

library(sqldf)

# Segmentando clientes de 2015

clientes_2015 = sqldf("SELECT ID_cliente,
                               MIN(dias_desde) AS 'recencia',
                               MAX(dias_desde) AS 'primeira_compra',
                               COUNT() AS 'frequencia',
                               AVG(quantia_compra) AS 'quantia'
                               FROM dados GROUP BY 1")

Loading required package: gsubfn

Loading required package: proto

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: RSQLite



### Visualizando a entidade chamada “clientes_2015” através de um print.

In [ ]:
print(clientes_2015)

      ID_cliente recencia primeira_compra frequencia     quantia
1             10     3829            3829          1   30.000000
2             80      343            3751          7   71.428571
3             90      758            3783         10  115.800000
4            120     1401            1401          1   20.000000
5            130     2970            3710          2   50.000000
6            160     2963            3577          2   30.000000
7            190     2211            3690          5   68.000000
8            220     2058            3664          2   25.000000
9            230     3985            3985          1   50.000000
10           240      463            3815          4   16.250000
11           260     3710            3710          1   15.000000
12           280     2065            3720          2   20.000000
13           290      787            3720          7   77.857143
14           320     3739            3739          1   15.000000
15           330     1167

### Definindo a classificação de clientes do ano de 2015 de acordo com frequência e quantidade de compras.

In [ ]:
clientes_2015$segment = 'NA'

clientes_2015$segment[which(clientes_2015$recencia  > 3653)] = 'inativo'

clientes_2015$segment[which(clientes_2015$recencia <= 3653 & clientes_2015$recencia > 3652)] = 'frio'

clientes_2015$segment[which(clientes_2015$recencia <= 3652 & clientes_2015$recencia > 3651)] = 'quente'

clientes_2015$segment[which(clientes_2015$recencia <= 365)] ='ativo'

clientes_2015$segment[which(clientes_2015$segment == 'quente' & clientes_2015$primeira_compra <= 3652)] = 'novo quente'

clientes_2015$segment[which(clientes_2015$segment == 'quente' & clientes_2015$quantia < 100)] = 'quente baixo valor'

clientes_2015$segment[which(clientes_2015$segment == 'quente' & clientes_2015$quantia >= 100)] ='quente alto valor'

clientes_2015$segment[which(clientes_2015$segment == 'ativo' & clientes_2015$primeira_compra <= 365)] = 'novo ativo'

clientes_2015$segment[which(clientes_2015$segment == 'ativo' & clientes_2015$quantia < 100)] = 'ativo baixo valor'

clientes_2015$segment[which(clientes_2015$segment == 'ativo' & clientes_2015$quantia >= 100)] = 'ativo alto valor'

clientes_2015$segment = factor(x = clientes_2015$segment, levels = c('inativo', 'frio','quente alto valor', 'quente baixo valor', 'novo quente','ativo alto valor', 'ativo baixo valor', 'novo ativo'))

### Definindo a classificação de clientes do ano de 2014 de acordo com frequência e quantidade de compras.

In [ ]:
# Segmentando clientes de 2014
clientes_2014 = sqldf("SELECT ID_cliente,
                               MIN(dias_desde) AS 'recencia',
                               MAX(dias_desde) AS 'primeira_compra',
                               COUNT() AS 'frequencia',
                               AVG(quantia_compra) AS 'quantia'
                               FROM dados GROUP BY 1")

In [ ]:
clientes_2014$segment = 'NA'

clientes_2014$segment[which(clientes_2014$recencia  > 3653)] = 'inativo'

clientes_2014$segment[which(clientes_2014$recencia <= 3653 & clientes_2014$recencia > 3652)] = 'frio'

clientes_2014$segment[which(clientes_2014$recencia <= 3652 & clientes_2014$recencia > 3651)] = 'quente'

clientes_2014$segment[which(clientes_2014$recencia <= 365)] ='ativo'

clientes_2014$segment[which(clientes_2014$segment == 'quente' & clientes_2014$primeira_compra <= 3652)] = 'novo quente'

clientes_2014$segment[which(clientes_2014$segment == 'quente' & clientes_2014$quantia < 100)] = 'quente baixo valor'

clientes_2014$segment[which(clientes_2014$segment == 'quente' & clientes_2014$quantia >= 100)] ='quente alto valor'

clientes_2014$segment[which(clientes_2014$segment == 'ativo' & clientes_2014$primeira_compra <= 365)] = 'novo ativo'

clientes_2014$segment[which(clientes_2014$segment == 'ativo' & clientes_2014$quantia < 100)] = 'ativo baixo valor'

clientes_2014$segment[which(clientes_2014$segment == 'ativo' & clientes_2014$quantia >= 100)] = 'ativo alto valor'

clientes_2014$segment = factor(x = clientes_2014$segment, levels = c('inativo', 'frio','quente alto valor', 'quente baixo valor', 'novo quente','ativo alto valor', 'ativo baixo valor', 'novo ativo'))

### Unindo clientes de 2014 e 2015 através dos IDs dos clientes em uma nova entidade chamada “novos_dados”.

In [ ]:
# computando a matriz de transição

novos_dados = merge(x = clientes_2014, y = clientes_2015, by = 'ID_cliente', all.x = TRUE)

head(novos_dados)

,ID_cliente,recencia.x,primeira_compra.x,frequencia.x,quantia.x,segment.x,recencia.y,primeira_compra.y,frequencia.y,quantia.y,segment.y
,<int>,<dbl>,<dbl>,<int>,<dbl>,<fct>,<dbl>,<dbl>,<int>,<dbl>,<fct>
1,10,3829,3829,1,30.00000,inativo,3829,3829,1,30.00000,inativo
2,80,343,3751,7,71.42857,ativo baixo valor,343,3751,7,71.42857,ativo baixo valor
3,90,758,3783,10,115.80000,NA,758,3783,10,115.80000,NA
4,120,1401,1401,1,20.00000,NA,1401,1401,1,20.00000,NA
5,130,2970,3710,2,50.00000,NA,2970,3710,2,50.00000,NA
6,160,2963,3577,2,30.00000,NA,2963,3577,2,30.00000,NA


### Criando uma tabela somatória das frequências de dados similares em uma tabela com nome de “transicao” e imprimindo-a com o comando print(). 

In [ ]:
​transition = table(novos_dados$segment.x,novos_dados$segment.y)

print(transition)

ERROR: ignored

### Somando o valor de todas as linhas.

In [ ]:
# Dividindo cada linha por sua soma
transicao = transicao / rowSums(transicao)
print(transicao)

ERROR: ignored

### Gerando previsão de clientes até 2025 com o laço for que multiplica o valor atual pela taxa de transição gerada através da variável “transicao”. 

In [ ]:
# usando a matriz de transição para previsão

#Iniciando a matriz com o número de clientes em cada segmento e depois de 10 ciclos

segments = matrix(nrow = 8, ncol = 11)

segments[, 1] = table(clientes_2015$segment)

colnames(segments) = 2015:2025

row.names(segments) = levels(clientes_2015$segment)

print(segments)

 

# Computando para cada  período

for (i in 2:11) {

   segments[, i] = segments[, i-1] %*% transicao

}

### Atribuindo uma receita anual para cada categoria de clientes e imprimindo o valor na tela através do comando print(). 

In [ ]:
# plotando os clientes inativos, ativos e ativos de alto valor

barplot(segments[1, ])

barplot(segments[2, ])


# mostrando como segmentos evoluem com o tempo

print(round(segments))

#Calculando o Costumer life time value / valor de tempo como cliente (CLV)

# Receita por segmento

# Aqui, tentaremos colocar o valor em dólares em nosso banco de dados atribuindo

# receita a alto valor ativo (US$ 323), baixo valor ativo (US$ 52) e

# novos clientes ativos (US$ 79), pois outros segmentos de clientes não contribuíram

# para a receita. Usaremos a receita média por segmento ativo de

# clientes para avaliar o banco de dados de clientes.

receita_anual = c(0, 0, 0, 0, 0, 323.57, 52.31, 79.17)

print(receita_anual)

### Calculando a estimativa de faturamento total ao multiplicar a receita pela quantidade de incidências.

In [ ]:
# Computando receita por ano

receita_segmento = receita_anual * segments

print(receita_segmento)

### Somando todos os segmentos processados com função colSums() para gerar a receita anual e imprimi-la em gráfico.

In [ ]:
# Computando a receita anual

receita_anual = colSums(receita_segmento)

print(round(receita_anual))

barplot(receita_anual)

Somando e arredondando os valores para gerar receita final acumulada por ano.

In [ ]:
# Calculando a receita acumulada

receita_acumulada = cumsum(receita_anual)

print(round(receita_acumulada))

barplot(receita_acumulada)

Gerando uma taxa de desconto anual crescente.

In [ ]:
# Criando o fator de desconto anual de 10%

taxa_desconto = 0.10

desconto = 1 / ((1 + taxa_desconto) ^ ((1:11) - 1))

print(desconto)

### Computando a taxa de desconto na receita anual e imprimindo visualmente.

In [ ]:
# Computando a receita anual descontada

desc_receita_anual = receita_anual * desconto

print(round(desc_receita_anual))

barplot(desc_receita_anual)

lines(receita_anual)

### Normalizando e arredondando a receita acumulada com desconto e gerando um gráfico.

In [ ]:
# Computando a receita acumulada

disc_receita_acumulada = cumsum(desc_receita_anual)

print(round(disc_receita_acumulada))

barplot(disc_receita_acumulada)

Subtraindo a receita de 2015 da receita de 2025 para entender seu valor.

In [ ]:
# Quanto vale nossa base de clientes?

print(disc_receita_acumulada[11] - receita_anual[1])